# 自然语言处理 - 通用信息抽取
<br>
<hr>

# 1. 实验介绍

## 1.1 实验背景

信息抽取旨在从非结构化自然语言文本中提取结构化知识，如实体、关系、事件等。

本实验使用预训练好的关系抽取模型，进行针对性的微调，以此不同环境下的测试任务

## 1.2 实验要求

a) 从数据集中选取合适的提示词和文本输入，

b) 加载合适的预训练模型并进行针对性微调。

c) 导出为 onnx 格式模型，并使用 TPU-MLIR 转化为 bmodel 模型。

## 1.3 实验环境

可以使用 Numpy 库进行相关数值运算，使用 PyTorch Transformers 库进行模型微调和加载预训练模型等。

## 1.4 注意事项
+ Python 与 Python Package 的使用方式，可在右侧 `API文档` 中查阅。
+ 当右上角的『Python 3』长时间指示为运行中的时候，造成代码无法执行时，可以重新启动 Kernel 解决（左上角『Kernel』-『Restart Kernel』）。

## 1.5 参考资料

Numpy：https://www.numpy.org/
    
Pytorch：https://pytorch.org/

transformers： https://huggingface.co/docs/transformers/index

## 1.6 文件说明 

```

│   ├── checkpoint # 微调保存下来的模型 
│   └── convert # 存放转换sh脚本文件和中间模型，将模型从onnx转化为bmodel文件的地方
│       ├── mlir2bmodel.sh # 将模型从mlir转换为bmodel的脚本文件
│       ├── onnx2mlir.sh # 将模型从onnx转换为mlir的脚本文件
│   └── ernie.py # 定义ernie模型
│   └── evaluate.py # 评估函数，提交后的推理方法就是依据evaluate.py文件
│   └── export_model.py # 导出模型为onnx格式
│   └── finetune.py # 模型微调训练
│   └── main.ipynb # 主notebook
│   └── model.py # 定义UIE模型
│   └── tools.py # 工具函数
│
│   ├── datasets/6434c6eaaad2f9ce44d79682-momodel # 数据集和预训练模型权重
│   └── data # 数据集
│       ├── competition_train.txt  # 训练集txt，content为句子内容（第一个输入），promt为提示词（第二个输入），result_list为输出，包括输出文本和起始位置与终止位置
│       ├── competition_valid.txt  # 验证集txt
│   └── uie_nano_pytorch # 预训练nano模型
│   └── uie_mini_pytorch # 预训练mini模型
│   └── uie_micro_pytorch # 预训练micro模型
```

# 2.实验内容

## 2.1 导入相关库，并进行参数设置

In [1]:
# 导入相关库
import argparse
import shutil
import sys
import time
import os
import torch
from itertools import chain
from typing import List, Union
from pathlib import Path
import numpy as np
from torch.utils.data import DataLoader
from transformers import (BertTokenizer, PreTrainedModel,
                          PreTrainedTokenizerBase, BertTokenizerFast)

from tools import IEDataset, logger, tqdm, set_seed, SpanEvaluator, EarlyStopping, logging_redirect_tqdm, logger
from model import UIE
from evaluate import evaluate

2023-05-16 15:14:16.112010: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic library 'libcudart.so.10.1'; dlerror: libcudart.so.10.1: cannot open shared object file: No such file or directory
2023-05-16 15:14:16.112057: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
# 复制模型权重到本地目录
!cp -r datasets/6434c6eaaad2f9ce44d79682-momodel/uie_nano_pytorch ./

# 这里仅用 nano 模型演示，也可使用 mini 或者 micro 模型
# !cp -r datasets/6434c6eaaad2f9ce44d79682-momodel/uie_mini_pytorch ./
# !cp -r datasets/6434c6eaaad2f9ce44d79682-momodel/uie_micro_pytorch ./

In [3]:
# 设置相关参数
class args:
    batch_size = 16
    learning_rate = 1e-5
    train_path = 'datasets/6434c6eaaad2f9ce44d79682-momodel/data/competition_train.txt'
    dev_path = 'datasets/6434c6eaaad2f9ce44d79682-momodel/data/competition_valid.txt'
    save_dir = 'uie_nano_pytorch'
    max_seq_len = 512
    num_epochs = 100
    seed = 42
    logging_steps = 100
    valid_steps = 100
    device = 'cpu'
    model = 'uie_nano_pytorch'
    max_model_num = 5

## 2.2 导入预训练模型

在下面的代码单元将实现以下功能：             
- 设置随机数种子
- 设置分词器，导入预训练模型
- 设置是否使用 gpu

In [6]:
# 设置随机数种子
set_seed(args.seed)
show_bar = True

# 设置分词器，导入预训练模型
tokenizer = BertTokenizerFast.from_pretrained(args.model)
model = UIE.from_pretrained(args.model)

# 设置是否使用gpu
if args.device == 'gpu':
    model = model.cuda()

In [7]:
# 查看UIE模型结构
model

UIE(
  (encoder): ErnieModel(
    (embeddings): ErnieEmbeddings(
      (word_embeddings): Embedding(40000, 312, padding_idx=0)
      (position_embeddings): Embedding(2048, 312)
      (token_type_embeddings): Embedding(4, 312)
      (task_type_embeddings): Embedding(16, 312)
      (LayerNorm): LayerNorm((312,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): ErnieEncoder(
      (layer): ModuleList(
        (0): ErnieLayer(
          (attention): ErnieAttention(
            (self): ErnieSelfAttention(
              (query): Linear(in_features=312, out_features=312, bias=True)
              (key): Linear(in_features=312, out_features=312, bias=True)
              (value): Linear(in_features=312, out_features=312, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): ErnieSelfOutput(
              (dense): Linear(in_features=312, out_features=312, bias=True)
              (LayerNorm): La

## 2.3 构建训练数据集

In [8]:
# 导入微调数据集和验证数据集
train_ds = IEDataset(args.train_path, tokenizer=tokenizer,
                     max_seq_len=args.max_seq_len)
dev_ds = IEDataset(args.dev_path, tokenizer=tokenizer,
                   max_seq_len=args.max_seq_len)
train_data_loader = DataLoader(
    train_ds, batch_size=args.batch_size, shuffle=True)
dev_data_loader = DataLoader(
    dev_ds, batch_size=args.batch_size, shuffle=True)

In [9]:
# 查看loader中的一个batch
input_ids, token_type_ids, att_mask, start_ids, end_ids = iter(train_data_loader).next()
print(input_ids)
print(input_ids.shape)

tensor([[   1,  417,  389,  ...,    0,    0,    0],
        [   1,   21,  139,  ...,    0,    0,    0],
        [   1,  593,  123,  ...,    0,    0,    0],
        ...,
        [   1,   31,  180,  ...,    0,    0,    0],
        [   1,   59,  247,  ...,    0,    0,    0],
        [   1,  450, 1140,  ...,    0,    0,    0]])
torch.Size([16, 512])


## 2.4 设置AdamW优化器，BCE损失以及评价指标

In [10]:
optimizer = torch.optim.AdamW(
    lr=args.learning_rate, params=model.parameters())

criterion = torch.nn.functional.binary_cross_entropy
metric = SpanEvaluator()

## 2.5 训练前的参数设置

In [11]:
# 训练前的参数初始化置零
loss_list = []
loss_sum = 0
loss_num = 0
global_step = 0
best_step = 0
best_f1 = 0
tic_train = time.time()
epoch_iterator = range(1, args.num_epochs + 1)

## 2.6 正式训练

由于在 notebook 中的内存限制，因此你需要使用 GPU Job 进行模型训练。

In [ ]:
if show_bar:
    train_postfix_info = {'loss': 'unknown'}
    epoch_iterator = tqdm(
        epoch_iterator, desc='Training', unit='epoch')

# 正式开始训练
for epoch in epoch_iterator:
    train_data_iterator = train_data_loader
    if show_bar:
        train_data_iterator = tqdm(train_data_iterator,
                                   desc=f'Training Epoch {epoch}', unit='batch')
        train_data_iterator.set_postfix(train_postfix_info)

    # 迭代训练集
    for batch in train_data_iterator:
        if show_bar:
            epoch_iterator.refresh()

        # 取出每一个batch的输入输出
        input_ids, token_type_ids, att_mask, start_ids, end_ids = batch

        # 如果使用gpu，则将其放入到cuda中
        if args.device == 'gpu':
            input_ids = input_ids.cuda()
            token_type_ids = token_type_ids.cuda()
            att_mask = att_mask.cuda()
            start_ids = start_ids.cuda()
            end_ids = end_ids.cuda()

        # 模型推理预测
        outputs = model(input_ids=input_ids,
                        token_type_ids=token_type_ids,
                        attention_mask=att_mask)
        start_prob, end_prob = outputs[0], outputs[1]

        # 进行反向传播与loss计算
        start_ids = start_ids.type(torch.float32)
        end_ids = end_ids.type(torch.float32)
        loss_start = criterion(start_prob, start_ids)
        loss_end = criterion(end_prob, end_ids)
        loss = (loss_start + loss_end) / 2.0
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        loss_list.append(float(loss))
        loss_sum += float(loss)
        loss_num += 1

        if show_bar:
            loss_avg = loss_sum / loss_num
            train_postfix_info.update({
                'loss': f'{loss_avg:.5f}'
            })
            train_data_iterator.set_postfix(train_postfix_info)

        global_step += 1
        if global_step % args.logging_steps == 0:
            time_diff = time.time() - tic_train
            loss_avg = loss_sum / loss_num

            if show_bar:
                with logging_redirect_tqdm([logger.logger]):
                    logger.info(
                        "global step %d, epoch: %d, loss: %.5f, speed: %.2f step/s"
                        % (global_step, epoch, loss_avg,
                           args.logging_steps / time_diff))
            else:
                logger.info(
                    "global step %d, epoch: %d, loss: %.5f, speed: %.2f step/s"
                    % (global_step, epoch, loss_avg,
                       args.logging_steps / time_diff))
            tic_train = time.time()

        # 迭代到一定次数后，对验证集进行评估
        if global_step % args.valid_steps == 0:
            save_dir = os.path.join(
                args.save_dir, "model_%d" % global_step)
            if not os.path.exists(save_dir):
                os.makedirs(save_dir)
            model_to_save = model
            model_to_save.save_pretrained(save_dir)
            tokenizer.save_pretrained(save_dir)
            if args.max_model_num:
                model_to_delete = global_step-args.max_model_num*args.valid_steps
                model_to_delete_path = os.path.join(
                    args.save_dir, "model_%d" % model_to_delete)
                if model_to_delete > 0 and os.path.exists(model_to_delete_path):
                    shutil.rmtree(model_to_delete_path)

            dev_loss_avg, precision, recall, f1 = evaluate(
                model, metric, data_loader=dev_data_loader, device=args.device, loss_fn=criterion)

            if show_bar:
                train_postfix_info.update({
                    'F1': f'{f1:.3f}',
                    'dev loss': f'{dev_loss_avg:.5f}'
                })
                train_data_iterator.set_postfix(train_postfix_info)
                with logging_redirect_tqdm([logger.logger]):
                    logger.info("Evaluation precision: %.5f, recall: %.5f, F1: %.5f, dev loss: %.5f"
                                % (precision, recall, f1, dev_loss_avg))
            else:
                logger.info("Evaluation precision: %.5f, recall: %.5f, F1: %.5f, dev loss: %.5f"
                            % (precision, recall, f1, dev_loss_avg))

            # 如果模型F1指标最优，那么保存该模型
            # Save model which has best F1
            if f1 > best_f1:
                if show_bar:
                    with logging_redirect_tqdm([logger.logger]):
                        logger.info(
                            f"best F1 performence has been updated: {best_f1:.5f} --> {f1:.5f}"
                        )
                else:
                    logger.info(
                        f"best F1 performence has been updated: {best_f1:.5f} --> {f1:.5f}"
                    )
                best_f1 = f1
                save_dir = os.path.join(args.save_dir, "model_best")
                model_to_save = model
                model_to_save.save_pretrained(save_dir)
                tokenizer.save_pretrained(save_dir)
            tic_train = time.time()

Training:   0% ░░░░░░░░░░  0/100 ?epoch/s eta ?
Training Epoch 1:   0% ░░░░░░░░░░  0/19 ?batch/s eta ?
Training:   0% ░░░░░░░░░░  0/100 ?epoch/s eta ? eta ?/home/jovyan/.virtualenvs/basenv/lib/python3.7/site-packages/transformers/modeling_utils.py:811: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  "The `device` argument is deprecated and will be removed in v5 of Transformers.", FutureWarning


# 3 模型评估和迁移

## 3.1 导出模型
将模型从pth格式导出为onnx格式，再将模型从onnx格式导出为bmodel格式

由于我们最终的模型需要在算能的BM1684X芯片上运行，BM1684X需要接受bmodel文件来运行指令集
我们使用算能提供的[TPU-MLIR](https://tpumlir.org/docs/quick_start/index.html)

具体步骤为
1. 使用torch.onnx.export导出onnx格式文件
2. 使用model_transform.py命令，将onnx文件转换为mlir中间文件
3. 使用model_deploy.py命令，将mlir中间文件转化为bmodel文件

关于TPU-MLIR的更多介绍，model_transform model_deploy命令的传入参数说明，请参考
https://tpumlir.org/docs/quick_start/index.html
  

## 3.2 torch转onnx

In [12]:
# 首先将模型导出为onnx格式
def export_onnx(output_path: Union[Path, str], tokenizer: PreTrainedTokenizerBase, model: PreTrainedModel, device: torch.device, input_names: List[str], output_names: List[str]):
    with torch.no_grad():
        model = model.to(device)
        model.eval()
        model.config.return_dict = True
        model.config.use_cache = False

        output_path = Path(output_path)

        # Create folder
        if not output_path.exists():
            output_path.mkdir(parents=True)
        save_path = output_path / "inference.onnx"

        dynamic_axes = {name: {0: 'batch', 1: 'sequence'}
                        for name in chain(input_names, output_names)}

        # Generate dummy input
        batch_size = 2
        seq_length = 6
        dummy_input = [" ".join([tokenizer.unk_token])
                       * seq_length] * batch_size
        inputs = dict(tokenizer(dummy_input, return_tensors="pt"))

        if save_path.exists():
            logger.warning(f'Overwrite model {save_path.as_posix()}')
            save_path.unlink()

        torch.onnx.export(model,
                          (inputs,),
                          save_path,
                          input_names=input_names,
                          output_names=output_names,
                          dynamic_axes=dynamic_axes,
                          do_constant_folding=True,
                          opset_version=11
                          )

    if not os.path.exists(save_path):
        logger.error(f'Export Failed!')

    return save_path

In [14]:
input_names = [
    'input_ids',
    'token_type_ids',
    'attention_mask',
]
output_names = [
    'start_prob',
    'end_prob'
]
output_path = 'uie_nano_pytorch'
model_path = 'uie_nano_pytorch'

tokenizer = BertTokenizer.from_pretrained(model_path)
model = UIE.from_pretrained(model_path)
device = torch.device('cpu')

save_path = export_onnx(output_path, tokenizer, model, device, input_names, output_names)

## 3.3 onnx转bmodel
打开一个命令行终端，切换到convert目录下

1.   

```bash
cd convert
```

2.   

```bash
cp ../uie_nano_pytorch/inference.onnx ./ 
```

3. 使用 model_transform.py 命令，将 onnx 文件转换为 mlir 中间文件
在命令行输入：

```bash
sh onnx2mlir.sh
```

4. 使用 model_deploy.py 命令，将 mlir 文件转换为 mlir 中间文件
在命令行输入：

```bash
sh mlir2bmodel.sh
```

## 3.4 提交代码示例

提交代码中，不需要撰写评估函数，但是需要提供原始的 `torch` 模型路径和转换后的 `bmodel` 模型路径。以 `uie-nano` 为例：

In [ ]:
bmodel_name = 'convert/uie-nano.bmodel'
model_name = 'uie_nano_pytorch'

# 4. 提交内容清单

- 转换的 bmodel 模型文件
- 包含`bmodel_name`和`model_name`的 main.py 文件
- 其他相关模型权重及文件

**注：模型测试过程不使用 bmodel 文件，使用原始模型进行推理，推理设备为 CPU**